# Miniming whatsapp chat with Koldo

In [105]:
import re
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join
import pandas as pd
import emoji
import re
import numpy as np
from stop_words import get_stop_words 
from ftfy import fix_text
from json import load
import itertools

In [106]:
f = open('chat.txt', 'r')
# Feed the file text into findall(); it returns a list of all the found strings
strings = re.findall('.(\d.\d.\d+\s+\d+:+\d+:+\d+).\s+(\w+\s\w+\s\w+|\w+\s\w+|\w+):\s(.*)', f.read())
f.close()

In [107]:
list_messages = []
for message in strings:
    a = dict({'TimeStamp': message[0], 'Sender_name': message[1], 'Content': message[2]})
    list_messages.append(a)

In [125]:
clean_list = []
number_words = []

for message in list_messages: # iterate over dictionaries of recipes
    
    # If there is a content key...
    if 'Content' in message.keys():
        # Look in the content key
        clean_msg = fix_text(message['Content'])
        
        if '<' in clean_msg and 'omitid' in clean_msg:
            if 'audio' in clean_msg: clean_msg = '<audio>'
            if 'video' in clean_msg: clean_msg = '<video>'
            if 'imagen' in clean_msg: clean_msg = '<imagen>'
            
        # Convert letters to lower case
        clean_msg = clean_msg.lower()
        
        for char in clean_msg:            
            # Remove special characters (excluding emoji, spaces : or _, which appear in the emoji)
            if not(char.islower() or char in [' ','_',':',"'",'<','>'] or char in emoji.UNICODE_EMOJI): 
                clean_msg = clean_msg.replace(char, "")
            
            # If emoji, add a space before the emoji
            if char in emoji.UNICODE_EMOJI:
                clean_msg  = emoji.demojize(clean_msg.replace(char,' ' + char))
        
        # Create a list of words
        number_words.append(len(clean_msg.split()))
        clean_list.append(clean_msg.split()) 

In [126]:
# Convert list of dictionaries to dataframe
df_msg = pd.DataFrame(list_messages)

# Filter rows of messages with content
content_true = [isinstance(elem, str) for elem in df_msg['Content']]
df_msg = df_msg[content_true].reset_index()

# Take only names and time
df_msg = df_msg[['Sender_name', 'TimeStamp']]

# Fix bad unicode characters in sender and change time format
#df_msg['TimeStamp']   = df_msg['TimeStamp'].apply(datetime.datetime.fromtimestamp)
df_msg['Sender_name'] = df_msg['Sender_name'].apply(fix_text)
df_msg['number_words'] = number_words
df_msg.head(10)

,Sender_name,TimeStamp,number_words
0,Paula AAFermin Cueto,1/7/16 21:27:44,1
1,Paula AAFermin Cueto,1/7/16 21:27:48,6
2,Paula AAFermin Cueto,1/7/16 21:28:05,8
3,Paula AAFermin Cueto,1/7/16 21:28:08,4
4,Paula AAFermin Cueto,1/7/16 21:28:11,2
5,Koldo,1/7/16 22:24:21,1
6,Koldo,1/7/16 22:24:28,2
7,Koldo,1/7/16 22:24:34,4
8,Paula AAFermin Cueto,2/7/16 16:23:30,2
9,Koldo,2/7/16 16:52:30,1


In [128]:
# Repeat row as many times as words in the the message
df_msg_rep = df_msg.set_index(['Sender_name'])['TimeStamp'].repeat(df_msg['number_words']).reset_index()

df_msg_rep['word']  = list(itertools.chain(*clean_list))
df_msg_rep['emoji'] = [':' in word for word in df_msg_rep['word']]
df_msg_rep.head()

,Sender_name,TimeStamp,word,emoji
0,Paula AAFermin Cueto,1/7/16 21:27:44,<imagen>,False
1,Paula AAFermin Cueto,1/7/16 21:27:48,mira,False
2,Paula AAFermin Cueto,1/7/16 21:27:48,el,False
3,Paula AAFermin Cueto,1/7/16 21:27:48,perfil,False
4,Paula AAFermin Cueto,1/7/16 21:27:48,de,False


In [129]:
df_msg_rep.to_csv('whatsapp_chat.csv', sep = '\t')